# Note Book For Seniment Analysis - EHmin

# Build DataFrame

 - for now, I use only 10000 data set

In [1]:
import pandas as pd 
df = pd.read_csv('review.csv')

In [2]:
df.head(5)

,stars,text
0,3,"If you decide to eat here, just be aware it is..."
1,5,I've taken a lot of spin classes over the year...
2,3,Family diner. Had the buffet. Eclectic assortm...
3,5,"Wow! Yummy, different, delicious. Our favo..."
4,4,Cute interior and owner (?) gave us tour of up...


In [3]:
df = df.rename(columns={'stars':'label','text':'text' })

In [4]:
small_df = df[0:10000]
small_df

,label,text
0,3,"If you decide to eat here, just be aware it is..."
1,5,I've taken a lot of spin classes over the year...
2,3,Family diner. Had the buffet. Eclectic assortm...
3,5,"Wow! Yummy, different, delicious. Our favo..."
4,4,Cute interior and owner (?) gave us tour of up...
...,...,...
9995,5,Excellent food and service. The place is funct...
9996,5,Just about to get tucked into a meatloaf that ...
9997,5,Outstanding customer service! And my car is dr...
9998,5,I and my husband went here for Dinner one day ...


In [5]:
small_df['label'] = small_df['label'] - 1
small_df

C:\Users\김동준\AppData\Local\Temp\ipykernel_5268\2213859284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['label'] = small_df['label'] - 1


,label,text
0,2,"If you decide to eat here, just be aware it is..."
1,4,I've taken a lot of spin classes over the year...
2,2,Family diner. Had the buffet. Eclectic assortm...
3,4,"Wow! Yummy, different, delicious. Our favo..."
4,3,Cute interior and owner (?) gave us tour of up...
...,...,...
9995,4,Excellent food and service. The place is funct...
9996,4,Just about to get tucked into a meatloaf that ...
9997,4,Outstanding customer service! And my car is dr...
9998,4,I and my husband went here for Dinner one day ...


In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(small_df , test_size=0.2, random_state=42)

In [7]:
from datasets import Dataset

small_train_dataset = Dataset.from_pandas(train_df)
small_eval_dataset = Dataset.from_pandas(test_df)
small_train_dataset[100]

{'label': 0,
 'text': 'I went to the drive thru hopefully I can get served so was at the speaker for half hour and lady had a attitude cause we asked if we could get service to say they had to change till and several cars left in the process I had to go home hungry that was totally sad luckily I not a diabetic',
 '__index_level_0__': 6998}

# Tokenize

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Train

In [9]:
# !pip install transformers

In [10]:
#!pip install --user --upgrade tensorflow

In [11]:
#!pip install --user --upgrade protobuf

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [13]:
#!pip install protobuf==3.20

In [14]:
#pip install tensorboardX

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

# Evaluate

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

# Trainer

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
trainer.train()

C:\anaconda\anaconda3\envs\transformer\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
